In [1]:
import os
from tqdm import tqdm
import pandas as pd
import xml.etree.ElementTree as ET
import numpy as np
import cv2
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam, RMSprop
import tensorflow as tf

import sys

sys.path.append("../")
from obj_detection import model

In [2]:
print(
    "GPU is available" if tf.test.gpu_device_name() else "GPU is NOT available"
)
if tf.test.gpu_device_name():
    physical_devices = tf.config.list_physical_devices("GPU")
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

GPU is NOT available


In [3]:
annotations_path = "../data/annotations/"
images_path = "../data/images/"

In [4]:
annotations = os.listdir(annotations_path)
images = os.listdir(images_path)

In [5]:
img_name_list = []
width_list = []
height_list = []
label_list = []
x_list = []
y_list = []
w_list = []
h_list = []

In [6]:
for i in tqdm(range(len(annotations))):
    tree = ET.parse(os.path.join(annotations_path, annotations[i]))
    root = tree.getroot()
    img_name = root.find("filename").text

    size = root.find("size")
    width = int(size.find("width").text)
    height = int(size.find("height").text)

    for group in root.findall("object"):
        label = group.find("name").text
        bbox = group.find("bndbox")
        xmin = int(bbox.find("xmin").text)
        ymin = int(bbox.find("ymin").text)
        xmax = int(bbox.find("xmax").text)
        ymax = int(bbox.find("ymax").text)
        x = int((xmin + xmax) / 2)
        y = int((ymin + ymax) / 2)
        w = abs(xmax - xmin)
        h = abs(ymax - ymin)

        img_name_list.append(img_name)
        width_list.append(width)
        height_list.append(height)
        x_list.append(x)
        y_list.append(y)
        w_list.append(w)
        h_list.append(h)
        label_list.append(label)

100%|██████████| 877/877 [00:00<00:00, 7312.82it/s]


In [7]:
dataset = pd.DataFrame(
    {
        "img_name": img_name_list,
        "width": width_list,
        "height": height_list,
        "x": x_list,
        "y": y_list,
        "w": w_list,
        "h": h_list,
        "label": label_list,
    }
)
dataset.head()

,img_name,width,height,x,y,w,h,label
0,road0.png,267,400,153,147,110,170,trafficlight
1,road1.png,400,283,206,172,104,218,trafficlight
2,road10.png,400,267,175,133,138,260,trafficlight
3,road100.png,400,385,199,165,328,321,speedlimit
4,road101.png,400,200,293,100,197,187,speedlimit


In [8]:
classes = dataset["label"].unique().tolist()
classes

['trafficlight', 'speedlimit', 'crosswalk', 'stop']

In [9]:
label_encoder = LabelBinarizer()

In [10]:
images = []
labels = []
bboxes = []
for index, row in dataset.iterrows():
    img_path = os.path.join(images_path, row["img_name"])
    img = cv2.imread(img_path)
    x_scale = 224.0 / int(row["width"])
    y_scale = 224.0 / int(row["height"])
    img = cv2.resize(img, (224, 224))
    images.append(img)

    x = int(row["x"] * x_scale) / 224.0
    y = int(row["y"] * y_scale) / 224.0
    w = int(row["w"] * x_scale) / 224.0
    h = int(row["h"] * y_scale) / 224.0
    bboxes.append((x, y, w, h))
    labels.append(row["label"])

In [11]:
images = np.array(images, dtype="float32") / 255.0
labels = np.array(labels)
bboxes = np.array(bboxes, dtype="float32")
labels = label_encoder.fit_transform(labels)

In [12]:
labels

array([[0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       ...,
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0]])

In [13]:
(
    train_images,
    val_images,
    train_bboxes,
    val_bboxes,
    train_labels,
    val_labels,
) = train_test_split(images, bboxes, labels, test_size=0.20, random_state=12)

In [14]:
losses = {
    "class_label": "categorical_crossentropy",
    "bounding_box": "mean_squared_error",
}

train_targets = {
    "class_label": train_labels,
    "bounding_box": train_bboxes,
}

val_targets = {
    "class_label": val_labels,
    "bounding_box": val_bboxes,
}

In [15]:
base_learning_rate = 0.00015

# opt = RMSprop(learning_rate=base_learning_rate)
opt = Adam(base_learning_rate)

In [16]:
ssd = model.SSD_Model(
    num_classes=len(classes),
    input_size=(224, 224, 3),
    optimizer=opt,
    losses=losses,
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_images (InputLayer)      [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['input_images[0][0]']           
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 224, 224, 64  36928       ['block1_conv1[0][0]']           
                                )                                                             

In [17]:
ssd.model_fit(
    train_images=train_images,
    train_targets=train_targets,
    val_images=val_images,
    val_targets=val_targets,
    epochs=15,
    batch_size=50,
)

Epoch 1/15
20/20 [==============================] - 62s 3s/step - loss: 0.9748 - bounding_box_loss: 0.0273 - class_label_loss: 0.9475 - bounding_box_accuracy: 0.5286 - class_label_accuracy: 0.6523 - val_loss: 0.6687 - val_bounding_box_loss: 0.0158 - val_class_label_loss: 0.6528 - val_bounding_box_accuracy: 0.5663 - val_class_label_accuracy: 0.7992
Epoch 2/15
20/20 [==============================] - 59s 3s/step - loss: 0.6122 - bounding_box_loss: 0.0119 - class_label_loss: 0.6004 - bounding_box_accuracy: 0.6643 - class_label_accuracy: 0.7769 - val_loss: 0.5717 - val_bounding_box_loss: 0.0112 - val_class_label_loss: 0.5605 - val_bounding_box_accuracy: 0.6707 - val_class_label_accuracy: 0.7791
Epoch 3/15
20/20 [==============================] - 68s 3s/step - loss: 0.4587 - bounding_box_loss: 0.0087 - class_label_loss: 0.4500 - bounding_box_accuracy: 0.7116 - class_label_accuracy: 0.8342 - val_loss: 0.5435 - val_bounding_box_loss: 0.0106 - val_class_label_loss: 0.5329 - val_bounding_box_ac

In [18]:
ssd.model.save("../models/output/model.h5")
ssd.model.save_weights("../models/output/model_weight.h5")